In [1]:
import pandas as pd
import os
import requests
import datetime as dt
from config import noaa_token as token

In [2]:
#variables
base = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/'
header = {'token':token,
          'Content-Type':'application/json'}

state_ids = {'PA':'FIPS:42',
            'OH':'FIPS:39',
            'MI':'FIPS:26',
            'IL':'FIPS:17',
            'WI':'FIPS:55',
            'MN':'FIPS:27',
            'IA':'FIPS:18',
            'NY':'FIPS:36'}

variables = {'ice_thinkness':'THIC',
            'precipitation':'PRCP',
            }

In [3]:
#get count
def get_c(json):
    return json['metadata']['resultset']['count']

def gather(endpt,load):
    initial = requests.get(base+endpt,headers=header,params=load).json()
    count = get_c(initial)
    json = initial['results']
    offset = 1000
    while offset < count:
        load['offset'] = offset
        json += requests.get(base+endpt,headers=header,params=load).json()['results']
        offset += 1000
    return pd.DataFrame(json)

In [4]:
variables = gather('datatypes',{'limit':'1000'})

In [5]:
len(variables)

1528

In [6]:
variables['mindate'] = pd.to_datetime(variables['mindate'])
variables['maxdate'] = pd.to_datetime(variables['maxdate'])

In [7]:
v_mintimes = variables[variables['mindate']<='1972-12-19']
v_times = v_mintimes[v_mintimes['maxdate']>='2016-05-05']
len(v_times)

78

In [9]:
v_times.to_csv(os.path.join('..', 'data_files', 'variables.csv'))

In [13]:
v_chosen = v_times.set_index("id")
v_chosen = v_chosen.loc[["DT32", "DX32","PRCP","SNWD", "THIC", "TMIN", "WDMV"]]
v_chosen.head(10)

v_times.to_csv(os.path.join('..', 'data_files', 'variables.csv'))

,mindate,maxdate,name,datacoverage
id,,,,
DT32,1763-01-01,2019-12-01,Number days with minimum temperature less than...,1.0
DX32,1763-01-01,2019-12-01,Number days with maximum temperature < 32 F.,1.0
PRCP,1781-01-01,2020-01-08,Precipitation,1.0
SNWD,1857-01-18,2020-01-08,Snow depth,1.0
THIC,1926-03-17,2019-11-30,Thickness of ice on water,1.0
TMIN,1763-01-01,2020-01-08,Minimum temperature,1.0
WDMV,1859-10-01,2019-12-31,Total wind movement,1.0
